In [ ]:
from PIL import Image
import requests
import torch

from datasets import load_dataset

dataset = load_dataset('eduvedras/Img_Desc',split='test',trust_remote_code=True)

image = dataset[0]['Chart']
image

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

checkpoint = "google/pix2struct-textcaps-base"
from transformers import AutoProcessor
from transformers import Pix2StructForConditionalGeneration, Pix2StructProcessor

processor = Pix2StructProcessor.from_pretrained(checkpoint)
inputs = processor(images=image, return_tensors="pt").to(device)

flattened_patches = inputs.flattened_patches
attention_mask = inputs.attention_mask

In [ ]:
from huggingface_hub import InferenceClient
from transformers import AutoModelForCausalLM

model_templates = Pix2StructForConditionalGeneration.from_pretrained("eduvedras/pix2struct-textcaps-base-desc-templates-final").to(device)
model_vars = Pix2StructForConditionalGeneration.from_pretrained("eduvedras/pix2struct-textcaps-base-desc-vars-final").to(device)

In [ ]:
generated_template_ids = model_templates.generate(flattened_patches=flattened_patches, attention_mask=attention_mask, max_length=193)
generated_template = processor.batch_decode(generated_template_ids, skip_special_tokens=True)[0]
print(generated_template)

generated_vars_ids = model_vars.generate(flattened_patches=flattened_patches, attention_mask=attention_mask, max_length=167)
generated_vars = processor.batch_decode(generated_vars_ids, skip_special_tokens=True)[0]

print(generated_vars)

In [ ]:
result = generated_template.replace("[]","[" + generated_vars + "]")

print(result)